In [26]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import re
import random

df = pd.read_csv("https://raw.githubusercontent.com/FarelN/Sistem-Rekomendasi/main/CSV/tugas%20sistem%20rekomendasi.csv")
df.head()

,Name,Address,Description
0,Farrel Nathanael,"Jl.Puri Sartika Blok B Semarang, Indonesia","Algoritma dan pemrograman, bahasa pemrograman,..."
1,PT Ruang Raya Indonesia,"Jl. Dr. Saharjo No.161, Manggarai Selatan, Teb...",Data & Visualization with Tableau Memahami car...
2,PT Nurul Fikri Cipta Inovasi,Jl. Situ Indah No.116 RT. 006 RW. 010 Kel. Tug...,Soft Skill. UI/UX. Design Pattern. Pemrograman...
3,PT. Sintesis Komunikasi Indonesia,"Jl. Cisanggiri V, No 6, Petogogan, Kebayoran B...",Logika dan Konsep Teknologi AI. Metode Penelit...
4,PT Neosia Pratama Indonusa,"Sona Topas Tower, lt. 5A/526. Jl. Jendral Sudi...",Pemodelan BIM Disiplin Struktur. Pemodelan BIM...


In [27]:
df.describe()

,Name,Address,Description
count,11,11,11
unique,11,11,11
top,Farrel Nathanael,"Jl.Puri Sartika Blok B Semarang, Indonesia","Algoritma dan pemrograman, bahasa pemrograman,..."
freq,1,1,1


1. Ikhtisar

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         11 non-null     object
 1   Address      11 non-null     object
 2   Description  11 non-null     object
dtypes: object(3)
memory usage: 392.0+ bytes


2. Tampilan Modul Pembelajaran (Sebelum Preprocessing)

In [29]:
def print_description(index):
    example = df[df.index == index][['Description', 'Name','Address']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Name:', example[1])
        print('Address:', example[2])

In [30]:
print_description(1)

Data & Visualization with Tableau Memahami cara untuk memvisualisasikan data menggunakan Tableau agar data dapat lebih mudah dipahami. Digital Product Analytics: memahami dasar-dasar digital product analytics untuk meningkatkan sistem operasional dalam bisnis. SQL for Analytics: Memahami dasar-dasar SQL dan mampu menggunakannya untuk ekstraksi data dari basis data relasional. Introduction to Data Analytics for Business: Menyusun, menyortir, memproses, dan mempelajari data bisnis menggunakan metode dan teknik tertentu untuk diubah menjadi suatu business insight. Final Project: mampu mengeksekusi suatu rangkaian projek data analitik yang mewakili real-case di industri meliputi tahap persiapan data, pengolahan data, pemodelan sampai dengan tahap visualisasi (dashboard). General Competencies: Pembelajaran tambahan yang dapat mendukung kemampuan peserta dalam mengikuti dan menerapkan program digital marketing yang meliputi: 1) Communication and Presentations SkilI 2)Analytical &amp; Strateg

In [31]:
print_description(3)

Logika dan Konsep Teknologi AI. Metode Penelitian AI. Siklus Projek AI. Pemrograman Python. Etika Profesi & Keterampilan Perusahaan. Proyek Akhir.
Name: PT. Sintesis Komunikasi Indonesia
Address: Jl. Cisanggiri V, No 6, Petogogan, Kebayoran Baru, Jakarta Selatan 12170


In [32]:
print_description(5)

Perancangan Produk/Jasa. Komunikasi. Leadership & Long Life Learning. Manajemen Organisasi Bisnis / Startup berbasis Teknologi Digital. Teknologi Informasi. Digital Mindset.
Name: PT Mitra Semeru Indonesia
Address: Jl. Mampang Prapatan Raya no 84A, Tegal Parang, Mampang Prapatan, Jakarta Selatan


3. Text Preprocessing

In [33]:
import nltk
nltk.download('stopwords')
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z #+_]')
stopworda = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    #text = ' '.join(word for word in text.split() if word not in stopworda) # hapus stopword dari kolom deskripsi
    return text
    
df['desc_clean'] = df['Description'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
df.head()

,Name,Address,Description,desc_clean
0,Farrel Nathanael,"Jl.Puri Sartika Blok B Semarang, Indonesia","Algoritma dan pemrograman, bahasa pemrograman,...",algoritma dan pemrograman bahasa pemrograman ...
1,PT Ruang Raya Indonesia,"Jl. Dr. Saharjo No.161, Manggarai Selatan, Teb...",Data & Visualization with Tableau Memahami car...,data visualization with tableau memahami cara...
2,PT Nurul Fikri Cipta Inovasi,Jl. Situ Indah No.116 RT. 006 RW. 010 Kel. Tug...,Soft Skill. UI/UX. Design Pattern. Pemrograman...,soft skill ui ux design pattern pemrograman mo...
3,PT. Sintesis Komunikasi Indonesia,"Jl. Cisanggiri V, No 6, Petogogan, Kebayoran B...",Logika dan Konsep Teknologi AI. Metode Penelit...,logika dan konsep teknologi ai metode peneliti...
4,PT Neosia Pratama Indonusa,"Sona Topas Tower, lt. 5A/526. Jl. Jendral Sudi...",Pemodelan BIM Disiplin Struktur. Pemodelan BIM...,pemodelan bim disiplin struktur pemodelan bim ...


4. Tampilan Modul Pembelajaran (Setelah Preprocessing)

In [35]:
def print_description_clean(index):
    example = df[df.index == index][['desc_clean', 'Name','Address']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Nama Perusahaan:', example[1])
        print('Alamat:', example[2])

In [36]:
print_description_clean(1)

data  visualization with tableau memahami cara untuk memvisualisasikan data menggunakan tableau agar data dapat lebih mudah dipahami digital product analytics memahami dasardasar digital product analytics untuk meningkatkan sistem operasional dalam bisnis sql for analytics memahami dasardasar sql dan mampu menggunakannya untuk ekstraksi data dari basis data relasional introduction to data analytics for business menyusun  menyortir  memproses  dan mempelajari data bisnis menggunakan metode dan teknik tertentu untuk diubah menjadi suatu business insight final project mampu mengeksekusi suatu rangkaian projek data analitik yang mewakili realcase di industri meliputi tahap persiapan data  pengolahan data  pemodelan sampai dengan tahap visualisasi  dashboard  general competencies pembelajaran tambahan yang dapat mendukung kemampuan peserta dalam mengikuti dan menerapkan program digital marketing yang meliputi 1  communication and presentations skili 2 analytical amp  strategic thinking skil

In [37]:
print_description_clean(3)

logika dan konsep teknologi ai metode penelitian ai siklus projek ai pemrograman python etika profesi  keterampilan perusahaan proyek akhir
Nama Perusahaan: PT. Sintesis Komunikasi Indonesia
Alamat: Jl. Cisanggiri V, No 6, Petogogan, Kebayoran Baru, Jakarta Selatan 12170


In [38]:
print_description_clean(5)

perancangan produk jasa komunikasi leadership  long life learning manajemen organisasi bisnis   startup berbasis teknologi digital teknologi informasi digital mindset
Nama Perusahaan: PT Mitra Semeru Indonesia
Alamat: Jl. Mampang Prapatan Raya no 84A, Tegal Parang, Mampang Prapatan, Jakarta Selatan


5. TF-IDF & Cosine Similarity

In [39]:
df.set_index('Name', inplace=True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['desc_clean'])
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim

array([[1.        , 0.1171141 , 0.09751346, 0.04595639, 0.01880241,
        0.02643456, 0.        , 0.        , 0.01660741, 0.01207077,
        0.        ],
       [0.1171141 , 1.        , 0.02914372, 0.01606104, 0.03436305,
        0.02600702, 0.00472925, 0.0071489 , 0.01469669, 0.01205545,
        0.01986504],
       [0.09751346, 0.02914372, 1.        , 0.02354804, 0.01712873,
        0.        , 0.        , 0.01399868, 0.        , 0.00942018,
        0.        ],
       [0.04595639, 0.01606104, 0.02354804, 1.        , 0.00768573,
        0.02641112, 0.        , 0.        , 0.00694445, 0.00422687,
        0.00959285],
       [0.01880241, 0.03436305, 0.01712873, 0.00768573, 1.        ,
        0.00791111, 0.        , 0.        , 0.0141509 , 0.00614921,
        0.        ],
       [0.02643456, 0.02600702, 0.        , 0.02641112, 0.00791111,
        1.        , 0.        , 0.        , 0.00867921, 0.01634321,
        0.        ],
       [0.        , 0.00472925, 0.        , 0.        , 0.

In [40]:
indices = pd.Series(df.index)
indices[:50]

0                           Farrel Nathanael
1                    PT Ruang Raya Indonesia
2               PT Nurul Fikri Cipta Inovasi
3          PT. Sintesis Komunikasi Indonesia
4                 PT Neosia Pratama Indonusa
5                  PT Mitra Semeru Indonesia
6     PT Bank Negara Indonesia (Persero) Tbk
7          PT Telkom Indonesia (Persero) Tbk
8        PT Abbauf Mulia Konsultan Teknologi
9                      Yayasan Hasnur Centre
10    PT Bank Rakyat Indonesia (Persero) Tbk
Name: Name, dtype: object

6. Modelling

In [41]:
def recommendations(name, cos_sim = cos_sim):
    
    recommended_modul = []
    
    # Mengambil nama modul berdasarkan variabel indicies
    idx = indices[indices == name].index[0]

    # Membuat series berdasarkan skor kesamaan
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    # mengambil index dan dibuat 3 baris rekomendasi terbaik
    top_10_indexes = list(score_series.iloc[1:4].index)
    
    for i in top_10_indexes:
        recommended_modul.append(list(df.index)[i])
        
    return recommended_modul

7. Prediksi

In [42]:
recommendations('Farrel Nathanael')

['PT Ruang Raya Indonesia',
 'PT Nurul Fikri Cipta Inovasi',
 'PT. Sintesis Komunikasi Indonesia']

In [43]:
recommendations('PT Ruang Raya Indonesia')

['Farrel Nathanael',
 'PT Neosia Pratama Indonusa',
 'PT Nurul Fikri Cipta Inovasi']

In [44]:
recommendations('PT Abbauf Mulia Konsultan Teknologi')

['PT Bank Rakyat Indonesia (Persero) Tbk',
 'Farrel Nathanael',
 'PT Ruang Raya Indonesia']